In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.6 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

NLP Deep Dive: RNNs

Text Preprocessing

Tokenization

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

"I know, it's a movie. But when it comes to portray real life (in any matter"

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#314) ['I','know',',','it',"'s",'a','movie','.','But','when','it','comes','to','portray','real','life','(','in','any','matter',')','it','should','be','as','faithful','as','possible','.','I'...]


In [7]:
first(spacy(['The U.S. dollar 1is1.00.']))

(#5) ['The','U.S.','dollar','1is1.00','.']

Unable to replicate the font. Probably in Latex.

Word Tokenization with fastai

In [8]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#374) ['xxbos','i','know',',','it',"'s",'a','movie','.','xxmaj','but','when','it','comes','to','portray','real','life','(','in','any','matter',')','it','should','be','as','faithful','as','possible','.'...]


In [9]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [10]:
coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Subword Tokenization

In [11]:
txts = L(o.open().read() for o in files[:2000])

In [12]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [13]:
subword(1000)

"▁I ▁know , ▁it ' s ▁a ▁movie . ▁But ▁when ▁it ▁come s ▁to ▁portray ▁real ▁life ▁( in ▁any ▁matter ) ▁it ▁should ▁be ▁as ▁f a i th ful ▁as ▁possibl e . ▁I ' m ▁sorry"

In [14]:
subword(200)

"▁I ▁ k n o w , ▁it ' s ▁a ▁movie . ▁B u t ▁w h en ▁it ▁co m es ▁to ▁p or t ra y ▁ re al ▁ li f e ▁( in ▁ an"

In [15]:
subword(10000)

'▁I ▁know , ▁it \' s ▁a ▁movie . ▁But ▁when ▁it ▁comes ▁to ▁portray ▁real ▁life ▁( in ▁any ▁matter ) ▁it ▁should ▁be ▁as ▁faithful ▁as ▁possible . ▁I \' m ▁sorry , ▁but ▁" E l ▁Mist'

In [16]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#374) ['xxbos','i','know',',','it',"'s",'a','movie','.','xxmaj','but','when','it','comes','to','portray','real','life','(','in','any','matter',')','it','should','be','as','faithful','as','possible','.'...]


In [17]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#374) ['xxbos','i','know',',','it',"'s",'a','movie','.','xxmaj'...]

In [18]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1968) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','to','of','is','it','in','i'...]"

In [19]:
nums = num(toks)[:20]; nums

TensorText([  2,  19, 108,  11,  17,  22,  12,  27,  10,   8,  28,  70,  17, 318,  14,   0, 190, 178,  40,  18])

In [20]:
' '.join(num.vocab[o] for o in nums)

"xxbos i know , it 's a movie . xxmaj but when it comes to xxunk real life ( in"